### Top Performing players
Load player static data from FPL and display top performing players

In [1]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlsxwriter

In [2]:
static_url = f'https://fantasy.premierleague.com/api/bootstrap-static/'
static_data = requests.get(static_url).json()

#### Load club and gameweek informations

In [3]:
position_mapping = {1:'Goalkeeper',2:'Defender',3:'Midfielder',4:'Forward'}
team_mapping = {}
gw_played = 0

for event in static_data['events']:
    if event['finished'] == True:
        gw_played = gw_played + 1

for team in static_data['teams']:
    team_mapping.update({team['code']:team['name']})


#### Load top performing goalkeepers

In [4]:
player_columns = ['Name','Club','Position','Cost','Points','Bonus','Goals','Assists','Saves','Penatly Saved %','CS','Points/Million','Points/Game','Minutes/Week','Ownership','YC','RC','Penalty Order']
player_dataframe = pd.DataFrame(columns = player_columns)

In [5]:
for element in static_data['elements']:
    position = position_mapping.get(element['element_type'])
    name = element['first_name'] +' '+ element['second_name']
    club = team_mapping.get(element['team_code'])
    cost = element['now_cost']/10
    points = element['total_points']
    bonus = element['bonus']
    ppm = points/cost
    ppg = element['points_per_game']
    mpw = element['minutes']/gw_played
    saves = element['saves']
    clean_sheets = element['clean_sheets']
    penalty_saved = element['penalties_saved']
    penalty_missed = element['penalties_missed']
    ownership = element['selected_by_percent']
    penalty_saved_pct = -1
    if penalty_saved + penalty_missed > 0:
        penalty_saved_pct = (element['penalties_saved']/(element['penalties_saved'] + element['penalties_missed'])) * 100
    goals = element['goals_scored']
    assists = element['assists']
    yc = element['yellow_cards']
    rc = element['red_cards']
    po = 0
    if element['penalties_order'] is not None:
        po = element['penalties_order']
    
    if mpw > 70:
        player_dataframe = player_dataframe.append(
        pd.Series([name,club,position,cost,points,bonus,goals,assists,saves,penalty_saved_pct,clean_sheets,ppm,ppg,mpw,ownership,
        yc,rc,po       
        ], index = player_columns
        ),ignore_index = True)        
        
player_dataframe['YC'] = player_dataframe['YC'].astype('int16')
player_dataframe['RC'] = player_dataframe['RC'].astype('int16')
player_dataframe['Penalty Order'] = player_dataframe['Penalty Order'].astype('int16')
player_dataframe['Goals'] = player_dataframe['Goals'].astype('int16')
player_dataframe['Assists'] = player_dataframe['Assists'].astype('int16')
player_dataframe['Points'] = player_dataframe['Points'].astype('int16')
player_dataframe['Bonus'] = player_dataframe['Bonus'].astype('int16')
player_dataframe['Saves'] = player_dataframe['Saves'].astype('int16')
player_dataframe['CS'] = player_dataframe['CS'].astype('int16')
player_dataframe['Penatly Saved %'] = player_dataframe['Penatly Saved %'].astype('float16')
player_dataframe['Points/Game'] = player_dataframe['Points/Game'].astype('float16')
player_dataframe['Ownership'] = player_dataframe['Ownership'].astype('float16')
#player_dataframe.info()

### Top 5 goalkeepers goalkeepers

In [6]:
pd.options.mode.chained_assignment = None
gk_dataframe = player_dataframe.loc[player_dataframe.Position == 'Goalkeeper' ]
gk_dataframe.sort_values('Points', ascending = False, inplace = True)
gk_dataframe[:5]

,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
83,Ederson Santana de Moraes,Man City,Goalkeeper,6.0,34,2,0,0,5,-1.0,5,5.666667,5.699219,90.0,10.000000,0,0,0
76,Alisson Ramses Becker,Liverpool,Goalkeeper,6.0,34,4,0,0,14,-1.0,4,5.666667,5.699219,90.0,8.703125,0,0,0
109,Hugo Lloris,Spurs,Goalkeeper,5.6,33,6,0,0,23,-1.0,3,5.892857,5.500000,90.0,9.703125,0,0,0
138,José Malheiro de Sá,Wolves,Goalkeeper,5.0,28,3,0,1,20,-1.0,2,5.600000,4.699219,90.0,2.599609,1,0,0
85,David de Gea,Man Utd,Goalkeeper,5.1,28,2,0,0,18,100.0,1,5.490196,4.000000,105.0,6.601562,0,0,0


### Top 5 goalkeepers with points to cost ratio

In [7]:
gk_dataframe.sort_values('Points/Million', ascending = False, inplace = True)
gk_dataframe[:5]

,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
109,Hugo Lloris,Spurs,Goalkeeper,5.6,33,6,0,0,23,-1.0,3,5.892857,5.500000,90.0,9.703125,0,0,0
11,David Raya Martin,Brentford,Goalkeeper,4.6,27,1,0,0,14,-1.0,3,5.869565,4.500000,90.0,10.000000,0,0,0
83,Ederson Santana de Moraes,Man City,Goalkeeper,6.0,34,2,0,0,5,-1.0,5,5.666667,5.699219,90.0,10.000000,0,0,0
76,Alisson Ramses Becker,Liverpool,Goalkeeper,6.0,34,4,0,0,14,-1.0,4,5.666667,5.699219,90.0,8.703125,0,0,0
138,José Malheiro de Sá,Wolves,Goalkeeper,5.0,28,3,0,1,20,-1.0,2,5.600000,4.699219,90.0,2.599609,1,0,0


### Top 3 goalkeepers with highest saves

In [8]:
gk_dataframe.sort_values('Saves', ascending = False, inplace = True)
gk_dataframe[:3]

,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
72,Illan Meslier,Leeds,Goalkeeper,5.0,13,0,0,0,27,-1.0,0,2.600000,2.199219,90.0,2.900391,1,0,0
109,Hugo Lloris,Spurs,Goalkeeper,5.6,33,6,0,0,23,-1.0,3,5.892857,5.500000,90.0,9.703125,0,0,0
60,Kasper Schmeichel,Leicester,Goalkeeper,5.1,20,2,0,0,21,-1.0,1,3.921569,3.300781,90.0,19.000000,0,0,0


### Top 3 goalkeepers with highest clean sheets

In [9]:
gk_dataframe.sort_values('CS', ascending = False, inplace = True)
gk_dataframe[:3]

,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
83,Ederson Santana de Moraes,Man City,Goalkeeper,6.0,34,2,0,0,5,-1.0,5,5.666667,5.699219,90.0,10.000000,0,0,0
76,Alisson Ramses Becker,Liverpool,Goalkeeper,6.0,34,4,0,0,14,-1.0,4,5.666667,5.699219,90.0,8.703125,0,0,0
11,David Raya Martin,Brentford,Goalkeeper,4.6,27,1,0,0,14,-1.0,3,5.869565,4.500000,90.0,10.000000,0,0,0


### Differential Goalkeeper 

##### Sorted by Points, Points/Million, Ownership

In [10]:
gk_dataframe.sort_values(['Points','Ownership'], ascending = [False,True], inplace = True)
print('Top Differential goalkeepers:')
print(gk_dataframe[['Name','Points','Points/Game','Cost','Ownership']][:5])

print('Goalkeepers with highest ownership:')
gk_dataframe.reset_index()
gk_dataframe.sort_values(['Ownership'], ascending = [False], inplace = True)
print(gk_dataframe[['Name','Points','Points/Game','Cost','Ownership']][:5])

Top Differential goalkeepers:
                          Name  Points  Points/Game  Cost  Ownership
76       Alisson Ramses Becker      34     5.699219   6.0   8.703125
83   Ederson Santana de Moraes      34     5.699219   6.0  10.000000
109                Hugo Lloris      33     5.500000   5.6   9.703125
138        José Malheiro de Sá      28     4.699219   5.0   2.599609
85                David de Gea      28     4.000000   5.1   6.601562
Goalkeepers with highest ownership:
                         Name  Points  Points/Game  Cost  Ownership
23             Robert Sánchez      19     3.199219   4.6  24.296875
60          Kasper Schmeichel      20     3.300781   5.1  19.000000
2           Emiliano Martínez      26     5.199219   5.5  14.296875
40              Edouard Mendy      28     5.601562   6.0  13.000000
83  Ederson Santana de Moraes      34     5.699219   6.0  10.000000


In [11]:
def_dataframe = player_dataframe.loc[player_dataframe.Position == 'Defender']
def_dataframe = def_dataframe[['Name','Club','Cost','Points','Bonus','CS','Goals','Assists','YC','RC','Penalty Order','Points/Million','Points/Game','Minutes/Week','Ownership']]

#### Top Defenders

In [12]:
def_dataframe.sort_values('Points', ascending = False, inplace = True)
def_dataframe[:8]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
82,João Pedro Cavaco Cancelo,Man City,6.1,44,6,5,0,2,0,0,0,7.213115,7.300781,90.000000,15.203125
84,Rúben Santos Gato Alves Dias,Man City,6.1,39,5,5,0,1,1,0,0,6.393443,6.500000,90.000000,27.796875
35,Marcos Alonso,Chelsea,6.0,39,4,4,1,1,2,0,0,6.500000,6.500000,89.166667,26.703125
15,Ethan Pinnock,Brentford,4.6,36,4,3,1,1,0,0,0,7.826087,6.000000,82.000000,6.000000
78,Trent Alexander-Arnold,Liverpool,7.5,35,8,3,0,2,0,0,0,4.666667,7.000000,75.000000,23.703125
74,Virgil van Dijk,Liverpool,6.6,34,4,4,0,1,0,0,0,5.151515,5.699219,90.000000,16.906250
38,Antonio Rüdiger,Chelsea,5.7,34,2,4,1,0,2,0,0,5.964912,5.699219,90.000000,20.406250
8,Pontus Jansson,Brentford,4.6,33,5,3,0,2,1,0,0,7.173913,5.500000,88.500000,8.203125


#### Top Defenders with points to cost ratio

In [13]:
def_dataframe.sort_values('Points/Million', ascending = False, inplace = True)
def_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
15,Ethan Pinnock,Brentford,4.6,36,4,3,1,1,0,0,0,7.826087,6.000000,82.000000,6.000000
82,João Pedro Cavaco Cancelo,Man City,6.1,44,6,5,0,2,0,0,0,7.213115,7.300781,90.000000,15.203125
8,Pontus Jansson,Brentford,4.6,33,5,3,0,2,1,0,0,7.173913,5.500000,88.500000,8.203125
19,Shane Duffy,Brighton,4.3,29,3,2,1,1,2,0,0,6.744186,4.800781,90.000000,23.093750
35,Marcos Alonso,Chelsea,6.0,39,4,4,1,1,2,0,0,6.500000,6.500000,89.166667,26.703125


#### Top Defenders with points/game

In [14]:
def_dataframe.reset_index()
def_dataframe.sort_values('Points/Game', ascending = False, inplace = True)
def_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
82,João Pedro Cavaco Cancelo,Man City,6.1,44,6,5,0,2,0,0,0,7.213115,7.300781,90.000000,15.203125
78,Trent Alexander-Arnold,Liverpool,7.5,35,8,3,0,2,0,0,0,4.666667,7.000000,75.000000,23.703125
35,Marcos Alonso,Chelsea,6.0,39,4,4,1,1,2,0,0,6.500000,6.500000,89.166667,26.703125
84,Rúben Santos Gato Alves Dias,Man City,6.1,39,5,5,0,1,1,0,0,6.393443,6.500000,90.000000,27.796875
15,Ethan Pinnock,Brentford,4.6,36,4,3,1,1,0,0,0,7.826087,6.000000,82.000000,6.000000


#### Top Defenders with attacking threat

In [15]:
def_dataframe['G+A'] = def_dataframe['Goals'] + def_dataframe['Assists']
def_dataframe.sort_values(['G+A','Points/Game','Points/Million'], ascending=[False,False,False],inplace=True)
#def_dataframe.sort_values(['Goals','Assists'], ascending = [False,False,False], inplace = True)
def_dataframe[['Name','Club','G+A','Goals','Assists','Points','Points/Game','Points/Million','Minutes/Week','Ownership']][:5]

,Name,Club,G+A,Goals,Assists,Points,Points/Game,Points/Million,Minutes/Week,Ownership
82,João Pedro Cavaco Cancelo,Man City,2,0,2,44,7.300781,7.213115,90.000000,15.203125
78,Trent Alexander-Arnold,Liverpool,2,0,2,35,7.000000,4.666667,75.000000,23.703125
35,Marcos Alonso,Chelsea,2,1,1,39,6.500000,6.500000,89.166667,26.703125
15,Ethan Pinnock,Brentford,2,1,1,36,6.000000,7.826087,82.000000,6.000000
3,Tyrone Mings,Aston Villa,2,0,2,28,5.601562,5.600000,75.000000,7.601562


#### Defenders with highest ownership

In [16]:
def_dataframe.sort_values(['Ownership','Points'], ascending = [False,True], inplace = True)
def_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership,G+A
89,Luke Shaw,Man Utd,5.4,20,2,1,0,1,2,0,0,3.703704,2.900391,95.500000,28.906250,1
84,Rúben Santos Gato Alves Dias,Man City,6.1,39,5,5,0,1,1,0,0,6.393443,6.500000,90.000000,27.796875,1
35,Marcos Alonso,Chelsea,6.0,39,4,4,1,1,2,0,0,6.500000,6.500000,89.166667,26.703125,2
78,Trent Alexander-Arnold,Liverpool,7.5,35,8,3,0,2,0,0,0,4.666667,7.000000,75.000000,23.703125,2
19,Shane Duffy,Brighton,4.3,29,3,2,1,1,2,0,0,6.744186,4.800781,90.000000,23.093750,2


In [17]:
mid_dataframe = player_dataframe.loc[player_dataframe.Position == 'Midfielder']
mid_dataframe = mid_dataframe[['Name','Club','Cost','Points','Bonus','CS','Goals','Assists','YC','RC','Penalty Order','Points/Million','Points/Game','Minutes/Week','Ownership']]

#### Top midfielders

In [18]:
mid_dataframe.sort_values(['Points','Points/Game'], ascending = [False,False], inplace = True)
mid_dataframe[:8]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
77,Mohamed Salah,Liverpool,12.6,57,8,4,5,3,1,0,2,4.523810,9.500000,90.000000,59.312500
124,Saïd Benrahma,West Ham,6.6,42,8,1,3,2,0,0,0,6.363636,7.000000,83.833333,37.406250
56,Abdoulaye Doucouré,Everton,5.6,42,7,2,2,4,3,0,0,7.500000,6.000000,105.000000,5.000000
58,Andros Townsend,Everton,5.5,40,5,2,3,2,1,0,4,7.272727,5.699219,86.500000,3.599609
117,Ismaila Sarr,Watford,6.3,39,7,0,4,0,0,0,3,6.190476,6.500000,90.000000,18.703125
90,Bruno Miguel Borges Fernandes,Man Utd,11.9,38,4,1,4,1,2,0,1,3.193277,5.398438,104.000000,27.093750
59,Demarai Gray,Everton,5.8,36,2,2,3,1,0,0,5,6.206897,5.101562,98.833333,19.093750
86,Paul Pogba,Man Utd,7.8,36,4,1,0,7,3,0,3,4.615385,5.101562,87.666667,22.093750


In [19]:
mid_dataframe.sort_values(['Points','Ownership'], ascending = [False,True], inplace = True)
mid_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
77,Mohamed Salah,Liverpool,12.6,57,8,4,5,3,1,0,2,4.523810,9.500000,90.000000,59.312500
56,Abdoulaye Doucouré,Everton,5.6,42,7,2,2,4,3,0,0,7.500000,6.000000,105.000000,5.000000
124,Saïd Benrahma,West Ham,6.6,42,8,1,3,2,0,0,0,6.363636,7.000000,83.833333,37.406250
58,Andros Townsend,Everton,5.5,40,5,2,3,2,1,0,4,7.272727,5.699219,86.500000,3.599609
117,Ismaila Sarr,Watford,6.3,39,7,0,4,0,0,0,3,6.190476,6.500000,90.000000,18.703125


In [20]:
mid_dataframe.sort_values('Points/Million', ascending = False, inplace = True)
mid_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
56,Abdoulaye Doucouré,Everton,5.6,42,7,2,2,4,3,0,0,7.500000,6.000000,105.000000,5.000000
58,Andros Townsend,Everton,5.5,40,5,2,3,2,1,0,4,7.272727,5.699219,86.500000,3.599609
124,Saïd Benrahma,West Ham,6.6,42,8,1,3,2,0,0,0,6.363636,7.000000,83.833333,37.406250
59,Demarai Gray,Everton,5.8,36,2,2,3,1,0,0,5,6.206897,5.101562,98.833333,19.093750
117,Ismaila Sarr,Watford,6.3,39,7,0,4,0,0,0,3,6.190476,6.500000,90.000000,18.703125


In [21]:
fwd_dataframe = player_dataframe.loc[player_dataframe.Position == 'Forward']
fwd_dataframe = fwd_dataframe[['Name','Club','Cost','Points','Bonus','CS','Goals','Assists','YC','RC','Penalty Order','Points/Million','Points/Game','Minutes/Week','Ownership']]

In [22]:
fwd_dataframe.sort_values(['Points','Points/Game'], ascending = [False,False], inplace = True)
fwd_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
122,Michail Antonio,West Ham,8.0,47,11,1,5,4,1,1,3,5.875000,9.398438,74.666667,45.312500
61,Jamie Vardy,Leicester,10.4,40,8,1,5,1,1,0,1,3.846154,6.699219,88.333333,13.101562
96,Allan Saint-Maximin,Newcastle,6.8,35,7,0,2,3,1,0,0,5.147059,5.800781,90.000000,16.593750
21,Neal Maupay,Brighton,6.5,31,5,1,4,0,1,0,2,4.769231,5.199219,82.500000,3.199219
10,Ivan Toney,Brentford,6.3,29,4,3,2,2,1,0,1,4.603175,4.800781,90.000000,14.898438


In [23]:
fwd_dataframe.sort_values(['Points','Ownership'], ascending = [False,True], inplace = True)
fwd_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
122,Michail Antonio,West Ham,8.0,47,11,1,5,4,1,1,3,5.875000,9.398438,74.666667,45.312500
61,Jamie Vardy,Leicester,10.4,40,8,1,5,1,1,0,1,3.846154,6.699219,88.333333,13.101562
96,Allan Saint-Maximin,Newcastle,6.8,35,7,0,2,3,1,0,0,5.147059,5.800781,90.000000,16.593750
21,Neal Maupay,Brighton,6.5,31,5,1,4,0,1,0,2,4.769231,5.199219,82.500000,3.199219
10,Ivan Toney,Brentford,6.3,29,4,3,2,2,1,0,1,4.603175,4.800781,90.000000,14.898438


In [24]:
fwd_dataframe.sort_values('Points/Million', ascending = False, inplace = True)
fwd_dataframe[:5]

,Name,Club,Cost,Points,Bonus,CS,Goals,Assists,YC,RC,Penalty Order,Points/Million,Points/Game,Minutes/Week,Ownership
122,Michail Antonio,West Ham,8.0,47,11,1,5,4,1,1,3,5.875000,9.398438,74.666667,45.312500
96,Allan Saint-Maximin,Newcastle,6.8,35,7,0,2,3,1,0,0,5.147059,5.800781,90.000000,16.593750
21,Neal Maupay,Brighton,6.5,31,5,1,4,0,1,0,2,4.769231,5.199219,82.500000,3.199219
10,Ivan Toney,Brentford,6.3,29,4,3,2,2,1,0,1,4.603175,4.800781,90.000000,14.898438
118,Emmanuel Dennis,Watford,5.3,23,3,1,2,1,3,0,0,4.339623,3.800781,80.166667,11.203125


In [25]:
max_cost = input('Enter maximum price: ')
position = input('Enter position (Goalkeeper,Defender,Midfielder,Forward): ')
#club = input('Enter club name')

player_dataframe.sort_values(['Points','Points/Game','Ownership'], ascending = [False,False,True], inplace = True)
player_dataframe[(player_dataframe['Cost'] <= float(max_cost)) & (player_dataframe['Position'] == position)][:5]

Enter maximum price: 5
Enter position (Goalkeeper,Defender,Midfielder,Forward): Forward


,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order


In [26]:
max_cost = input('Enter maximum price: ')
position = input('Enter position (Goalkeeper,Defender,Midfielder,Forward): ')
club =     input('Enter club name: ')

player_dataframe.sort_values(['Points','Points/Game','Ownership'], ascending = [False,False,True], inplace = True)
player_dataframe[(player_dataframe['Cost'] <= float(max_cost)) & (player_dataframe['Position'] == position) & (player_dataframe['Club'] == club )][:5]

Enter maximum price: 7
Enter position (Goalkeeper,Defender,Midfielder,Forward): Forward
Enter club name: Brentford


,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
10,Ivan Toney,Brentford,Forward,6.3,29,4,2,2,0,-1.0,3,4.603175,4.800781,90.0,14.898438,1,0,1


####  Top penalty takers

In [30]:
player_dataframe.sort_values(['Points','Points/Game','Ownership'], ascending = [False,False,True], inplace = True)
player_dataframe[player_dataframe['Penalty Order'] == 1][:7]

,Name,Club,Position,Cost,Points,Bonus,Goals,Assists,Saves,Penatly Saved %,CS,Points/Million,Points/Game,Minutes/Week,Ownership,YC,RC,Penalty Order
61,Jamie Vardy,Leicester,Forward,10.4,40,8,5,1,0,-1.0,1,3.846154,6.699219,88.333333,13.101562,1,0,1
90,Bruno Miguel Borges Fernandes,Man Utd,Midfielder,11.9,38,4,4,1,0,0.0,1,3.193277,5.398438,104.000000,27.093750,2,0,1
47,Wilfried Zaha,Crystal Palace,Midfielder,6.9,29,3,2,1,0,-1.0,2,4.202899,4.800781,90.000000,6.800781,1,0,1
10,Ivan Toney,Brentford,Forward,6.3,29,4,2,2,0,-1.0,3,4.603175,4.800781,90.000000,14.898438,1,0,1
69,Patrick Bamford,Leeds,Forward,7.8,21,2,1,2,0,-1.0,0,2.692308,4.199219,72.666667,5.898438,1,0,1
99,Teemu Pukki,Norwich,Forward,5.9,21,1,2,0,0,-1.0,0,3.559322,3.500000,82.333333,5.300781,0,0,1
105,James Ward-Prowse,Southampton,Midfielder,6.4,20,2,1,0,0,-1.0,2,3.125000,3.300781,90.000000,3.099609,1,0,1
